# Time-Series Modeling with VAE

This notebook demonstrates how to use the enhanced VAE model for time-series data analysis and generation. Unlike image VAEs that work with spatial patterns, time-series VAEs model sequential patterns in medical data.

## Key Concepts:
- **Sequence Encoding**: RNN-based encoder captures temporal dependencies
- **Latent Representation**: Compressed representation of patient trajectories
- **Sequence Generation**: RNN decoder reconstructs realistic medical sequences

## Applications:
- Patient trajectory modeling and generation
- Medical sequence anomaly detection
- Synthetic data generation for rare conditions
- Treatment pattern analysis

In [8]:
from pyhealth.datasets import split_by_visit, get_dataloader
from pyhealth.trainer import Trainer
from pyhealth.models import VAE
from pyhealth.datasets import SampleDataset

import torch
import numpy as np
import matplotlib.pyplot as plt

## Create Time-Series Medical Data

We'll create sample patient trajectories showing disease progression and treatment sequences.

In [9]:
# Create sample time-series medical data
ts_samples = [
    {
        "patient_id": "patient-0",
        "visit_id": "visit-0",
        "visits": ["diabetes", "metformin", "hba1c_test", "insulin"],
        "label": 1.0,
    },
    {
        "patient_id": "patient-1",
        "visit_id": "visit-1",
        "visits": ["hypertension", "lisinopril", "bp_check", "followup"],
        "label": 0.5,
    },
    {
        "patient_id": "patient-2",
        "visit_id": "visit-2",
        "visits": ["asthma", "albuterol", "peak_flow", "steroids"],
        "label": 0.8,
    },
    {
        "patient_id": "patient-3",
        "visit_id": "visit-3",
        "visits": ["depression", "sertraline", "therapy", "counseling"],
        "label": 0.3,
    },
]

# Create dataset
ts_dataset = SampleDataset(
    samples=ts_samples,
    input_schema={"visits": "sequence"},
    output_schema={"label": "regression"},
    dataset_name="timeseries_demo",
)

print("Time-series dataset created")
print(f"Number of samples: {len(ts_dataset)}")
print(f"Vocabulary size: {len(ts_dataset.input_processors['visits'].code_vocab)}")

Processing samples: 100%|██████████| 4/4 [00:00<00:00, 19949.13it/s]

Time-series dataset created
Number of samples: 4
Vocabulary size: 18


## Create and Train Time-Series VAE

The VAE will learn to encode patient trajectories into a latent space and reconstruct them.

In [10]:
# Create time-series VAE model
ts_model = VAE(
    dataset=ts_dataset,
    feature_keys=["visits"],
    label_key="label",
    mode="regression",
    input_type="timeseries",  # Key parameter for time-series mode
    hidden_dim=32,  # Smaller latent dimension for sequences
)

print("Time-series VAE created")
print(f"Input type: {ts_model.input_type}")
print(f"Has embedding model: {hasattr(ts_model, 'embedding_model')}")
print(f"Has RNN encoder: {hasattr(ts_model, 'encoder_rnn')}")
print(f"Latent dimension: {ts_model.hidden_dim}")

Time-series VAE created
Input type: timeseries
Has embedding model: True
Has RNN encoder: True
Latent dimension: 32


## Understanding the Time-Series VAE Architecture

The time-series VAE differs from image VAEs:

1. **EmbeddingModel**: Converts categorical sequences to dense vectors
2. **RNN Encoder**: Processes sequential embeddings, capturing temporal patterns
3. **Latent Space**: Fixed-size representation of the entire sequence
4. **Linear Decoder**: Reconstructs the sequence's compressed representation

This architecture can learn patterns like "diabetes → metformin → insulin" or "asthma → albuterol → steroids".

In [11]:
# Prepare data for training
train_dataloader = get_dataloader(ts_dataset, batch_size=1, shuffle=True)

# Create trainer
trainer = Trainer(
    model=ts_model, 
    device="cuda" if torch.cuda.is_available() else "cpu",
    metrics=["kl_divergence", "mse", "mae"]
)

# Train the model (reduced epochs for demo)
print("Training time-series VAE...")
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=train_dataloader,  # Using same data for demo
    epochs=5,
    monitor="kl_divergence",
    monitor_criterion="min",
    optimizer_params={"lr": 1e-3},
)

print("Training completed!")

VAE(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict(
    (visits): Embedding(18, 32, padding_idx=0)
  ))
  (encoder_rnn): GRU(32, 32, batch_first=True)
  (mu): Linear(in_features=32, out_features=32, bias=True)
  (log_std2): Linear(in_features=32, out_features=32, bias=True)
  (decoder_linear): Linear(in_features=32, out_features=32, bias=True)
)
Metrics: ['kl_divergence', 'mse', 'mae']
Device: cuda

Training time-series VAE...
Training:
Batch size: 1
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7a19402482c0>
Monitor: kl_divergence
Monitor criterion: min
Epochs: 5
Patience: None



Epoch 0 / 5: 100%|██████████| 4/4 [00:00<00:00, 249.98it/s]

--- Train epoch-0, step-4 ---
loss: 12.0682



Evaluation: 100%|██████████| 4/4 [00:00<00:00, 632.67it/s]

--- Eval epoch-0, step-4 ---
kl_divergence: 6.1801
mse: 0.0002
mae: 0.0102
loss: 13.0345
New best kl_divergence score (6.1801) at epoch-0, step-4



...

--- Eval epoch-3, step-16 ---
kl_divergence: 7.2321
mse: 0.0003
mae: 0.0119
loss: 13.5714




Epoch 4 / 5: 100%|██████████| 4/4 [00:00<00:00, 264.15it/s]

--- Train epoch-4, step-20 ---
loss: 15.0954



Evaluation: 100%|██████████| 4/4 [00:00<00:00, 602.67it/s]

--- Eval epoch-4, step-20 ---
kl_divergence: 6.8994
mse: 0.0003
mae: 0.0119
loss: 10.0336
Loaded best model
Training completed!


## Evaluate Reconstruction Performance

Check how well the VAE reconstructs the original sequences.

In [12]:
# Evaluate on training data
eval_results = trainer.evaluate(train_dataloader)
print("Evaluation Results:")
for metric, value in eval_results.items():
    print(f"{metric}: {value:.4f}")

# Get reconstruction examples
data_batch = next(iter(train_dataloader))
with torch.no_grad():
    output = ts_model(**data_batch)
    
print(f"\nReconstruction shape: {output['y_prob'].shape}")
print(f"Original shape: {output['y_true'].shape}")
print(f"Loss: {output['loss'].item():.4f}")

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 599.21it/s]

Evaluation Results:
kl_divergence: 8.0949
mse: 0.0003
mae: 0.0125
loss: 13.2461

Reconstruction shape: torch.Size([1, 32])
Original shape: torch.Size([1, 32])
Loss: 23.6351


## Generate New Medical Sequences

Sample from the latent space to generate new patient trajectories.

In [13]:
# Generate new sequences by sampling from latent space
ts_model.eval()
with torch.no_grad():
    # Sample random latent vectors
    latent_samples = torch.randn(3, ts_model.hidden_dim).to(ts_model.device)
    
    # Decode to get sequence representations
    generated_sequences = ts_model.decoder(latent_samples)
    
    print("Generated sequence representations:")
    print(f"Shape: {generated_sequences.shape}")
    print(f"Sample values: {generated_sequences[0, :5].cpu().numpy()}")
    
    # The generated sequences represent points in the embedded space
    # In a full implementation, you might use a decoder RNN to generate
    # actual token sequences, but here we show the latent generation concept

Generated sequence representations:
Shape: torch.Size([3, 32])
Sample values: [-1.181458   -0.8360508   0.49716952 -0.88951784  0.26240543]


## Key Insights

### How Time-Series VAE Works:
1. **Input Processing**: Categorical sequences are embedded using the EmbeddingModel
2. **Sequence Encoding**: RNN processes the embedded sequence to capture temporal patterns
3. **Latent Compression**: Variable-length sequences become fixed-size latent vectors
4. **Reconstruction**: Decoder attempts to recreate the embedded sequence representation

### Medical Applications:
- **Trajectory Analysis**: Understand typical patient progression patterns
- **Synthetic Data**: Generate realistic patient histories for research
- **Anomaly Detection**: Identify unusual treatment sequences
- **Outcome Prediction**: Learn sequence patterns that correlate with outcomes

### Differences from Image VAE:
- **Temporal vs Spatial**: Captures time-ordered dependencies instead of spatial patterns
- **Variable Length**: Handles sequences of different lengths
- **Categorical Data**: Works with medical codes, diagnoses, treatments
- **Generation**: Creates new realistic patient trajectories